In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
from PIL import Image
import torch.nn.functional as F
import numpy as np

In [2]:
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet,self).__init__()
        #INPUT CHANNEL, OUTPUT CHANNEL, KERNEL SIZE
        #INPUT SIZE = (32,32) OUTPUT SIZE= (28,28)
        self.conv1 = nn.Conv2d(1,6,5)
        self.conv2 = nn.Conv2d(6,16,5)
        #INPUT DIM, OUTPUT DIM
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84,10)
        
    def forward(self, x):
        #POOL SIZE = 2
        #INPUT SIZE = (28,28) OUTPUT SIZE = (14, 14) OUTPUT CHANBNEL = 6
        x = F.max_pool2d(F.relu(self.conv1(x)), 2)
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        
        #Flatten
        x = x.view(x.size()[0], -1)
        
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        
        x = self.fc3(x)
        
        return x

In [3]:
def load_data(train_batch_size, test_batch_size):
    # Fetch training data: total 60000 samples
    train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('data', train=True, download=True,
                       transform=transforms.Compose([
                           transforms.Resize((32, 32)),
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=train_batch_size, shuffle=True)

       # Fetch test data: total 10000 samples
    test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('data', train=False, transform=transforms.Compose([
            transforms.Resize((32, 32)),
            transforms.ToTensor(),
            transforms.Normalize((0.1307,), (0.3081,))
        ])),
        batch_size=test_batch_size, shuffle=True)

    return (train_loader, test_loader)

In [4]:
def train(model, optimizer, epoch, train_loader, log_interval):
    model.train()

    loss_fn = torch.nn.CrossEntropyLoss()

    #iterating over batches of data
    for batch_idx, (data,target) in enumerate(train_loader):
        data, target = Variable(data), Variable(target)

        #CLear gradients
        optimizer.zero_grad()

        #Foward propagation
        output = model(data)

        loss = loss_fn(output,target)

        loss.backward()

        #update bias and weight
        optimizer.step()

        if batch_idx % log_interval == 0:
            print('Train set, Epoch {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                       100. * batch_idx / len(train_loader),
                loss.data))

In [5]:
def test(model, epoch, test_loader):
    model.eval()

    test_loss = 0
    correct = 0

    loss_fn = torch.nn.CrossEntropyLoss(size_average=False)

    #Iterating over data
    for data, target in test_loader:
        data, target = Variable(data), Variable(target)

        output = model(data)

        test_loss += loss_fn(output, target).data

        pred = np.argmax(output.data, axis=1)

        correct = correct + np.equal(pred,target.data).sum()

        test_loss /= len(test_loader.dataset)
        print('\nTest set, Epoch {} , Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(epoch,
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [6]:
torch.manual_seed(123)

model = LeNet()

lr = 0.01
momentum = 0.5
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

train_batch_size = 64
test_batch_size = 1000
train_loader, test_loader = load_data(train_batch_size,test_batch_size)

epochs = 10
log_interval = 100

for epoch in range(1, epochs + 1):
    train(model, optimizer, epoch, train_loader, log_interval=log_interval)
    test(model, epoch, test_loader)

Train set, Epoch 1 [0/60000 (0%)]	Loss: 2.297229
Train set, Epoch 1 [6400/60000 (11%)]	Loss: 2.138054
Train set, Epoch 1 [12800/60000 (21%)]	Loss: 0.613613
Train set, Epoch 1 [19200/60000 (32%)]	Loss: 0.289441
Train set, Epoch 1 [25600/60000 (43%)]	Loss: 0.424027
Train set, Epoch 1 [32000/60000 (53%)]	Loss: 0.269818
Train set, Epoch 1 [38400/60000 (64%)]	Loss: 0.259016
Train set, Epoch 1 [44800/60000 (75%)]	Loss: 0.116685
Train set, Epoch 1 [51200/60000 (85%)]	Loss: 0.156493
Train set, Epoch 1 [57600/60000 (96%)]	Loss: 0.123462


/home/nilbson/anaconda3/lib/python3.7/site-packages/torch/nn/_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set, Epoch 1 , Average loss: 0.0122, Accuracy: 961/10000 (9%)


Test set, Epoch 1 , Average loss: 0.0145, Accuracy: 1920/10000 (19%)


Test set, Epoch 1 , Average loss: 0.0138, Accuracy: 2874/10000 (28%)


Test set, Epoch 1 , Average loss: 0.0118, Accuracy: 3840/10000 (38%)


Test set, Epoch 1 , Average loss: 0.0156, Accuracy: 4796/10000 (47%)


Test set, Epoch 1 , Average loss: 0.0110, Accuracy: 5765/10000 (57%)


Test set, Epoch 1 , Average loss: 0.0152, Accuracy: 6714/10000 (67%)


Test set, Epoch 1 , Average loss: 0.0104, Accuracy: 7684/10000 (76%)


Test set, Epoch 1 , Average loss: 0.0112, Accuracy: 8643/10000 (86%)


Test set, Epoch 1 , Average loss: 0.0106, Accuracy: 9601/10000 (96%)

Train set, Epoch 2 [0/60000 (0%)]	Loss: 0.124037
Train set, Epoch 2 [6400/60000 (11%)]	Loss: 0.055559
Train set, Epoch 2 [12800/60000 (21%)]	Loss: 0.259057
Train set, Epoch 2 [19200/60000 (32%)]	Loss: 0.111003
Train set, Epoch 2 [25600/60000 (43%)]	Loss: 0.076017
Train set, Epoch 2 [32000/60

Train set, Epoch 8 [6400/60000 (11%)]	Loss: 0.013216
Train set, Epoch 8 [12800/60000 (21%)]	Loss: 0.008795
Train set, Epoch 8 [19200/60000 (32%)]	Loss: 0.032473
Train set, Epoch 8 [25600/60000 (43%)]	Loss: 0.095979
Train set, Epoch 8 [32000/60000 (53%)]	Loss: 0.043767
Train set, Epoch 8 [38400/60000 (64%)]	Loss: 0.034108
Train set, Epoch 8 [44800/60000 (75%)]	Loss: 0.044857
Train set, Epoch 8 [51200/60000 (85%)]	Loss: 0.003593
Train set, Epoch 8 [57600/60000 (96%)]	Loss: 0.013817

Test set, Epoch 8 , Average loss: 0.0045, Accuracy: 988/10000 (9%)


Test set, Epoch 8 , Average loss: 0.0029, Accuracy: 1979/10000 (19%)


Test set, Epoch 8 , Average loss: 0.0026, Accuracy: 2969/10000 (29%)


Test set, Epoch 8 , Average loss: 0.0041, Accuracy: 3956/10000 (39%)


Test set, Epoch 8 , Average loss: 0.0048, Accuracy: 4943/10000 (49%)


Test set, Epoch 8 , Average loss: 0.0035, Accuracy: 5930/10000 (59%)


Test set, Epoch 8 , Average loss: 0.0024, Accuracy: 6920/10000 (69%)


Test set, Epoch 8 ,